# Reading LHE and HPMC Files

## Introduction

MadGraph outputs event files in Les Houches Accord Event File (LHE) format.  The MadGraph/ Pythia8 interface outputs an event file in HepMC format.  (Pythia8 is used to simulate emission of colored particles from the initial state protons.)  Both of these formats are described in more detail in the markdown (.md) files in this repository.

## Methods

### Preliminaries

In [3]:
from __future__ import print_function

### PDG Codes for Leptons

The dictionary below contains the particle codes used to identify leptons and antileptons.  
These numbers are described at http://pdg.lbl.gov/2007/reviews/montecarlorpp.pdf

In [4]:
lepton_pdg_id_dict = {
    'electron': 11,
    'positron': -11,
    'muon': 13,
    'antimuon': -13
}

### Method to read LHE file

This method reads an unzipped LHE file to obtain lepton data from four lepton events.  In general it will be called by read_4l_event_file, which handles unzipping the file when necessary and calls this method
when the target file has suffix ".lhe".

In [2]:
def read_4l_LHE_file(file_path_and_name, return_valid_event_list, final_state_dict):
    """ Reads LHE file specified by the string file_path_and_name.
        Returns a 20 column numpy array of valid 4 lepton events.
        If return_valid_event_list, we return a list of the indices of events read in that
        are valid four lepton events.
        final_state_dict contains descriptions of what the various final states
        ('4e', '2e2mu', or '4mu') are; this is set by other methods in this notebook
        """
        
    import numpy as np
    
    # fields containing relevant information in LHE convention
    PDG_ID_FIELD, E_FIELD, PX_FIELD, PY_FIELD, PZ_FIELD = 0, 9, 6, 7, 8
    
    number_of_columns = 20
    
    with open(file_path_and_name, 'r') as event_file:

        data_table = []
        
        if return_valid_event_list:
            i_event = 1 # event numbering starting from 1, not 0
            valid_event_list = []
        
        # read the LHE file and write the relevant info to data_table
        in_event = False    
        for line in event_file:
            if "<event>" in line:
                in_event = True
                event_momentum_dict = {}
            elif "</event>" in line:
                in_event = False
                valid_final_state = False
                event_lepton_id_dict = {lepton: len(event_momentum_dict[lepton]) 
                                       for lepton in event_momentum_dict}
                for final_state in final_state_dict:
                    if final_state_dict[final_state] == event_lepton_id_dict: 
                        valid_final_state = True
                        break
                if valid_final_state:
                    i_lepton = 0
                    event_row = [0 for i in range(number_of_columns)]
                    for lepton in event_momentum_dict:
                        for momentum in event_momentum_dict[lepton]:
                            event_row[i_lepton] = lepton_pdg_id_dict[lepton]
                            for i_p, p in enumerate(momentum):
                                event_row[4 * i_lepton + i_p] = p
                            i_lepton += 1
                    data_table.append(event_row)
                    if return_valid_event_list:
                        valid_event_list.append(i_event)
                        i_event += 1
            elif in_event:
                fields = line.split()
                for lepton, pdg_id in lepton_pdg_id_dict.items():
                    try:
                        line_id = int(fields[PDG_ID_FIELD]) 
                        if line_id == pdg_id:
                            momentum = [float(fields[n]) for n in 
                                        [E_FIELD, PX_FIELD, PY_FIELD, PZ_FIELD]]
                            if lepton in event_momentum_dict:
                                event_momentum_dict[lepton].append(momentum)
                            else:
                                event_momentum_dict[lepton] = [momentum]
                    except ValueError:
                        pass
                    
    # return numpy array with event information and, if requested, a list of valid events 
    if return_valid_event_list:
        return (np.array(data_table), valid_event_list)
    else:
        return np.array(data_table)

### Method to read HepMC file

This method reads an unzipped HepMC file to obtain lepton data from four lepton events.  In general it will be called by read_4l_event_file, which handles unzipping the file when necessary and calls this method
when the target file has suffix ".hepmc".

In [4]:
def read_4l_HEPMC_file(file_path_and_name, return_valid_event_list, final_state_dict):
     """ Reads HepMC file specified by the string file_path_and_name.
        If make_final_state_list we make a list of which final state 
        ('4e', '2e2mu', or '4mu') each event belongs to so that subsequent lists of events
        have the same order as our LHE file.
        final_state_dict contains descriptions of what the various final states
        ('4e', '2e2mu', or '4mu') are; this is set by other methods in this notebook
        """
           
    # fields containing relevant information in HepMC convention
    STATUS_field, PDG_ID_field = 8, 2
    E_FIELD, PX_FIELD, PY_FIELD, PZ_FIELD = 6, 3, 4, 5
    
    with open(file_path_and_name, 'r') as event_file:

        data_table = []
        
        if return_valid_event_list:
            i_event = 1 # event numbering starting from 1, not 0
            valid_event_list = []

        # read the HepMC file and put the relevant info in file_momentum_dict        
        first_event = True
        in_event = False
        for line in event_file:
            if "E" in line.strip()[:1]:
                in_event = True
                if first_event:
                    first_event = False
                else:
                    valid_final_state = False
                    event_lepton_id_dict = {lepton: len(event_momentum_dict[lepton])
                                            for lepton in event_momentum_dict}
                    for final_state in final_state_dict:
                        if final_state_dict[final_state] == event_lepton_id_dict: 
                            valid_final_state = True
                            break
                    if valid_final_state:
                        i_lepton = 0
                        event_row = [0 for i in range(number_of_columns)]
                        for lepton in event_momentum_dict:
                            for momentum in event_momentum_dict[lepton]:
                                event_row[i_lepton] = lepton_pdg_id_dict[lepton]
                                for i_p, p in enumerate(momentum):
                                    event_row[4 * i_lepton + i_p] = p
                            i_lepton += 1
                    data_table.append(event_row)
                    if return_valid_event_list:
                        valid_event_list.append(i_event)
                        i_event += 1
                event_momentum_dict = {}
            elif in_event:
                fields = line.split()
                if fields and fields[0] == 'P':
                    try:
                        status = int(fields[STATUS_field])
                        if status != 1: continue
                    except IndexError:
                        continue
                for lepton, pdg_id in lepton_pdg_id_dict.items():
                    try:
                        line_id = int(fields[PDG_ID_field])
                        if line_id == pdg_id:
                            momentum = [float(fields[n]) for n in 
                                        [E_FIELD, PX_FIELD, PY_FIELD, PZ_FIELD]]
                            if lepton in event_momentum_dict:
                                event_momentum_dict[lepton].append(momentum)
                            else:
                                event_momentum_dict[lepton] = [momentum]
                    except (IndexError, ValueError) as e:
                        pass
                    
    # return numpy array with event information and, if requested, a list of valid events
    if make_final_state_list:
        return (np.array(data_table), valid_event_list)
    else:
        return np.array(data_table)

### Method to Read Event File in LHE, HepMC, or MEKD format

This method reads a possibly zipped LHE, HepMC, or MEKD input file (for which we assume the suffixes "dat" or "txt"), returns the identity and momenta of the four leptons in four lepton events in a numpy array with shape (number_of_events, 20), and rezips the input file if it was initially zipped.

In [21]:
def read_4l_event_file(file_path_and_name, return_valid_event_list = False,
                      file_format = 'auto'):
    """ Reads a possibly zipped LHE, HepMC, or MEKD file located at file_path_and_name 
        and returns a numpy array with shape (number_of_events, 20) containing the
        lepton identities and momenta.
        If return_valid_event_list then a list of the event numbers which are
        valid four lepton events in the initial file is returned.
        If file_format = 'auto' the type of file to read is determined from the file
        extension after unzipping.  (MEKD input files are assumed to end in "dat" or "txt"
        as "mekd" is often used for MEKD output files.)
        Specifying (case insensitive) 'LHE', 'HepMC', or for the case of MEKD, 
        "dat", "txt", or "mekd", overrides this determination.  Method will return None 
        and warn the user if the correct file format cannot be determined.
    """
        
    import warnings
    import subprocess

    # dict with descriptions of final states
    final_state_dict = {
        '2e2mu': {
            'electron': 1,
            'positron': 1,
            'muon': 1,
            'antimuon': 1
        },
        '4e': {
            'electron': 2,
            'positron': 2,
        },
        '4mu': {
            'muon': 2,
            'antimuon': 2,            
        }
    }
        
    need_to_rezip = False
    if file_path_and_name[-3:] == '.gz':
        try:
            subprocess.call(['gunzip', file_path_and_name])
            file_path_and_name = file_path_and_name[:-3]
            need_to_rezip = True
        except OSError:
            warnings.warn("Failure to unzip " + file_path_and_name + 
                          ".  Probably gunzip is not installed.")
            return None

    # Determine file type
    if file_format == 'auto':
        format_determined = True
        for file_ext in ['lhe', 'hepmc', 'dat', 'txt']:
            if file_path_and_name.endswith(file_ext):
                file_format = file_ext
                format_undetermined = False
                break
        if format_undetermined:
            warnings.warn('File format of ' + file_path_and_name + 
                         ' could not be determined.')
            return None
    else:
        file_format = file_format.lower()
        
    # 
    if format == 'lhe':
        output = read_4l_LHE_file(file_path_and_name, return_valid_event_list, final_state_dict)
    elif file_format == 'hepmc':
        output = read_4l_HEPMC_file(file_path_and_name, return_valid_event_list, final_state_dict)
    elif file_format in ['dat', 'txt', 'mekd']:
        if return_valid_event_list:
            warnings.warn('return_valid_event_list option not implemented for MEKD input files.')
        output = np.fromfile(file_path_and_name, dtype = 'float64', sep = " ")
    else:
        warnings.warn('Could not understand file_format = ' + file_format)
        return None

    if need_to_rezip:
        subprocess.call(['gzip', file_path_and_name])
            
    return output

## Test

Thes tests assume that this notebook is the "processing_raw_data" or other subdirectory of "Machine-Learning-on-Four-Lepton-Events", and that appropriate event files are in the raw_data subdirectory of "Machine-Learning-on-Four-Lepton-Events".  If these assumptions are incorrect, modify the line below to point to the approprate event files for testing.

In [22]:
raw_data_path = "../raw_data/"

In [ ]:
lhe_test_path_and_name = raw_data_path + 
hepmc_test_path_and_name = raw_data_path +
mekd_test_path_and_name = raw_data_path + 

In [ ]:
for file_format, file_path_and_name in zip(['lhe', 'hepmc', 'mekd'], 
                                           [lhe_test_path_and_name, hepmc_test_path_and_name,
                                            mekd_test_path_and_name]):
    for return_valid_event_list in [True, False]:
        for file_format_argument in ['auto', ]read_4l_event_file(file_path_and_name, return_valid_event_list = False,
                      file_format = 'auto')

## Write Four-Momenta To MEKD-Style File

We will not write a separate method for writing the numpy array to MEKD-style input file as this can be easily achieved using numpy.ndarray.tofile().